In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import random
import datetime

import multiprocessing
from joblib import Parallel, delayed


import keras
#from keras import applications
#from keras.preprocessing.image import ImageDataGenerator
#from keras import optimizers
from keras.models import Sequential, Model 
from keras import layers
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
#import cv2
from sklearn.model_selection import train_test_split
import keras_metrics as km
  
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import f1_score
#from sklearn.metrics import cohen_kappa_score
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import multilabel_confusion_matrix


In [2]:

import tensorflow as tf



In [3]:

SensorPositonFile = 'D:\\SensorStatsSmall.csv'
folder = 'D:\\CraneData\\'
SaveModelFolder = 'D:\\SavedModel\\'

img_height , img_width = 3, 100
FrameLength = img_width
numberFrames = 1200
NumberOfFiles = 100
DataSmoothing = 1 # 0 = none, 1 = rolling average, 2 = rolling StdDev
num_cores = multiprocessing.cpu_count() -1

OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,dtype=int, missing_values=0)


In [4]:

def truthVector(Filename):
    # Parses the filename, and compares it against the record of sensor position on cranes
    # inputs: filename
    # outputs: truth vector


    #Parsing the file name.  Assuming it is in the standard format
    sSensor = Filename[23]
    sDate = datetime.datetime.strptime('20'+Filename[10:21],"%Y%m%d-%H%M")

    mask = []

    i=0
    #loops through the known sensor movements, and creates a filter mask
    for spf in OutputVectors:
        
        startDate = datetime.datetime.strptime(str(spf[0])+str(spf[1]).zfill(2)+str(spf[2]).zfill(2)
            +str(spf[3]).zfill(2)+str(spf[4]).zfill(2),"%Y%m%d%H%M")
        #datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
        endDate = datetime.datetime.strptime(str(spf[5])+str(spf[6]).zfill(2)+str(spf[7]).zfill(2)
            +str(spf[8]).zfill(2)+str(spf[9]).zfill(2),"%Y%m%d%H%M")
        #datetime.date(int(spf[5]), int(spf[6]), int(spf[7])) + datetime.timedelta(hours=spf[8]) + datetime.timedelta(minutes=spf[9])
        
        if sDate >= startDate and sDate <= endDate and int(spf[10]) == int(sSensor):
            mask.append(True)
            i+=1
        else:
            mask.append(False)
        
    if i != 1: print('error ', i, Filename)

    results = OutputVectors[mask,11:]

    if i > 1: 
        print('Found Two ', Filename)
        results = results[0,:]
    #np.array(results)

    return results


In [5]:

def makeFrames(input): #,sequ,frameLength):
    frames=[] #np.array([],dtype=object,)
    segmentGap = int((np.shape(input)[0]-FrameLength)/numberFrames)
    #print(segmentGap,sequ, frameLength)
    for i in range(numberFrames):
        start = i * segmentGap
        imageMatrix = input[start:start+FrameLength,:]
        np.matrix(imageMatrix)
        imageMatrix = imageMatrix.T
        frames.append(imageMatrix)
    
    return frames


In [15]:

def Smoothing(RawData, SmoothType = 1, SmoothDistance=15):

    if SmoothType == 0:
        SmoothedData = RawData
    elif SmoothType ==1:
        SmoothedData = RawData
        for i in range(SmoothDistance):
            for j in range(3):
                SmoothedData[j,i]=np.average(RawData[j,0:i])
        for i in range(np.shape(RawData)[0]-SmoothDistance):
            for j in range(3):
                SmoothedData[j,i+SmoothDistance]=np.average(RawData[j,i:i+SmoothDistance])


    return SmoothedData


In [7]:

def ParseFile(Filename):

    Results = truthVector(Filename)
    print("have results")
    fileData = np.genfromtxt(open(folder+Filename,'r'), delimiter=',',skip_header=0,missing_values=0).T[2:5,:]
    print("have file")
    smoothData = Smoothing(fileData)
    print("smoothed")
    #frames = makeFrames(smoothData.T) #,numberFrames,img_width)
    #frames = np.asarray(frames)
    
    return smoothData, Results


In [16]:

files = os.listdir(folder)
files = random.sample(files,NumberOfFiles)

print('Sample Created')

ResultsSet = np.zeros((len(files),np.shape(OutputVectors[:,11:])[1]))

i=0


Sample Created


In [9]:

Data = Parallel(n_jobs=num_cores)(delayed(ParseFile)(file) for file in files)


In [10]:

DataSet = [] 
i=0
for datum in Data:
    DataSet.append(datum[0])
    ResultsSet[i]=datum[1][0]
    i+=1

DataSet = np.asarray(DataSet)

print('Data Parsed')

#ResultsSet = ResultsSet[0:np.shape(DataSet)[0],:]

x_train, x_test, y_train, y_test = train_test_split(DataSet, ResultsSet, test_size=0.20, shuffle=True, random_state=0)


Data Parsed


In [11]:
np.shape(ResultsSet)

(100, 4)

In [12]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [13]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(np.shape(y_train)[1], activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [14]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)


IndexError: Exception encountered when calling layer "softmax" (type Softmax).

tuple index out of range

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • mask=None

In [ ]:

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]


In [ ]:

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)